# Decision Tree From Scratch Using If/Else Statements
This notebook will explore building a decision tree in the most basic way possible by using a series of if/else statements.

## Table of Contents

1. [Introduction](#1)
2. [Measuring Node Impurity](#2)
    1. [Gini Index](#3)
    1. [Entropy](#4)
3. [Classifying Predictions](#5)
4. [Building the Tree](#6)
    1. [Simulate Training Data](#7)
5. [Predicting New Observations](#8)
    1. [Simulate Testing Data](#9)
6. [References](#10)

<a id="1"></a> <br>
## Introduction
Many tutorials explain how to build a decision tree from scratch but often complicate the process. There are more optimal ways to build the model, however this notebook was created to increase interpretability. To make the process easier, both the inputs and outputs are binary. We will therefore be creating a classification decision tree. The notebook assumes the reader has basic knowledge of decision trees.

<a id="2"></a> <br>
## Measuring Node Impurity
In order to build the decision tree, there must be a way to evaluate which input is the best question to ask at the given time. At every node in the tree one must evaluate the quality of inputs at the current depth. One can evaluate the quality of the split using node impurity calculations. For classification, the two typical methods used to measure node impurity are Gini Index and Entropy.

<a id="3"></a> <br>
### Gini Index
Gini Index can be thought of as the degree or probability of a particular variable being wrongly classified when it is randomly chosen. [1](https://blog.quantinsti.com/gini-index/) Gini index varies between 0 and 1, where 0 indicates all elements belong to a certain class, and 1 indicates that the elements are randomly distributed between various classes. [1](https://blog.quantinsti.com/gini-index/) The function below takes four inputs, zero correct indicates the observations of class 0 that are correct, zero incorrect indicates the observations of class 0 that are incorrect. One correct indicates the observations of class 1 that are correct, one incorrect indicates the observations of class 1 that are incorrect. The gini index is calculated using the formula below, where c is the number of classes, and $p_{i}$ is the proportion of the samples that belong to class c for a particular node.

$$
\operatorname{Gini}=1-\sum_{i=1}^{C}\left(p_{i}\right)^{2}
$$

In [8]:
gini = function(zero_correct, zero_incorrect, one_correct, one_incorrect){
  
  gini_one = 1 - ((one_correct / (one_correct + one_incorrect))^2 + (one_incorrect / (one_correct + one_incorrect))^2)
  gini_zero = 1 - ((zero_correct / (zero_correct + zero_incorrect))^2 + (zero_incorrect / (zero_incorrect + zero_correct))^2)
  
  return((((one_correct + one_incorrect) / (one_correct + one_incorrect + zero_correct + zero_incorrect)) * gini_one) 
         + (((zero_correct + zero_incorrect) / (one_correct + one_incorrect + zero_correct + zero_incorrect)) * gini_zero))
}

<a id="4"></a> <br>
### Entropy
Entropy can be roughly thought of as the amount of variance within the data, which is the degree of uncertainty, impurity or disorder within the node. [1](https://blog.quantinsti.com/gini-index/) The entropy function created below takes the same inputs as the gini function.  The entropy is calculated using the formula below, where c is the number of classes, and $p_{i}$ is the proportion of the samples that belong to class c for a particular node.

$$
\text { Entropy }=\sum_{i=1}^{C}-p_{i} \times \log _{2}\left(p_{i}\right)
$$

In [9]:
entropy = function(zero_correct, zero_incorrect, one_correct, one_incorrect){
  
  entropy_one = - ((one_correct / (one_correct + one_incorrect)) * log2(one_correct / (one_correct + one_incorrect)) 
                    + (one_incorrect / (one_correct + one_incorrect)) * log2(one_incorrect / (one_correct + one_incorrect)))
  entropy_zero = - ((zero_correct / (zero_correct + zero_incorrect)) * log2(zero_correct / (zero_correct + zero_incorrect)) 
                     + (zero_incorrect / (zero_correct + zero_incorrect)) * log2(zero_incorrect / (zero_correct + zero_incorrect)))
  
  return(((one_correct + one_incorrect) / (one_correct + one_incorrect + zero_correct + zero_incorrect)) * entropy_one + 
           ((zero_correct + zero_incorrect) / (one_correct + one_incorrect + zero_correct + zero_incorrect)) * entropy_zero)
}

<a id="5"></a> <br>
### Classifying Predictions
A third function has been created to evaluate the mode of the observations in each leaf. This will be used for prediction purposes. When given testing data, the prediction will be computed as the observation that occurs most often within the given leaf of the tree built from the training data.

In [10]:
mode = function(data){
  unique_data = unique(data)
  return(unique_data[which.max(tabulate(match(data, unique_data)))])
}

<a id="6"></a> <br>
## Building the Tree
The following function will be used to create the classification decision tree model.

In [11]:
tree = function(training_data){

  # Number of observations
  n = dim(training_data)[1]
  
  # Number of inputs
  p = dim(training_data)[2] - 1
  
  # Create list of 0 & 1
  values = c(0, 1)
  
  # Repeats list of 0 & 1 values, p times
  values_p = lapply(numeric(p), function(x) values)
  
  # Finds each possible combination of 0 & 1 and stores it in a matrix
  mat = as.matrix(expand.grid(values_p))

  # Number of inputs at each depth of the tree
  num_inputs = c(p:1)
  
  # Initialize count for adding values to list
  count = 0
  
  # Create matrix for outputting results
  tree_output = matrix(0, nrow = dim(mat)[1], ncol = p+1)
  
  # Iterate through each row of matrix to try all possible splits
  for (b in c(1:dim(mat)[1])){
    
    # Restart data frame which will be broken down in the loop
    training_data_loop = data.frame(training_data)
    
    # Each row in the matrix indicates a way of getting to leaf
    for (i in c(1:dim(mat)[2])){
      
      # Create empty matrix to append gini values to
      gini_values = matrix(0, nrow = as.numeric(num_inputs[i]))
      
      # Loop through number of inputs
      for (j in c(1:num_inputs[i])){
        
        # Target variable = 0 (Assumes target variable is last column in data set)
        if (length(training_data_loop[training_data_loop[[ncol(training_data_loop)]] == 0, j]) != 0){
          zero_correct = mean(training_data_loop[training_data_loop[[ncol(training_data_loop)]] == 0, j])
        } else {
          zero_correct = 0
        }
        zero_incorrect = 1 - zero_correct
        
        # Target variable = 1 (Assumes target variable is last column in data set)
        if (length(training_data_loop[training_data_loop[[ncol(training_data_loop)]] == 1, j]) != 0){
          one_correct = mean(training_data_loop[training_data_loop[[ncol(training_data_loop)]] == 1, j]) 
        } else {
          one_correct = 0
        }
        one_incorrect = 1 - one_correct
        
        # Append gini to gini_values matrix
        gini_values[j] = gini(one_correct, one_incorrect, zero_correct, zero_incorrect)
        
        # Add 1 to count
        count = count + 1
        
      }
      
      # Add obsevation from matrix to tree output
      tree_output[b, j] = mat[b, i]
      
      # Removes input with best gini value because it will be used for the split
      training_data_loop = training_data_loop[training_data_loop[which.min(gini_values)] == mat[b, i], - which.min(gini_values)]
      
    }
    
    # Classify the output as the mode of the observations in the leaf
    tree_output[b, p+1] = mode(training_data_loop)
    
  }
  
  # Convert tree output to data frame
  tree_output = as.data.frame(tree_output)
  
  # Make column names the same as the column names from the original data frame
  colnames(tree_output) = head(names(training_data), n = p+1)
                    
  # Return the output of the tree
  return(tree_output)
  
}

<a id="7"></a> <br>
### Simulate Training Data
Training data will be simulated below to run through the tree function and create a tree with each possible combination of inputs and their predicted values.

In [12]:
# Number of observations/rows in data
n = 1000

# Simulate data, last column must be the target variable
exercises = sample(c(0,1), replace = TRUE, size = n)
drinks = sample(c(0,1), replace = TRUE, size = n)
smokes = sample(c(0,1), replace = TRUE, size = n)
relationship = sample(c(0,1), replace = TRUE, size = n)
heart_pain = sample(c(0,1), replace = TRUE, size = n)

# Create training data
training_data = as.data.frame(cbind(drinks, exercises, smokes, relationship, heart_pain))

# Test tree function
tree_output = tree(training_data)

# Print the output from the tree
print(tree_output)

   drinks exercises smokes relationship heart_pain
1       0         0      0            0          1
2       0         0      0            1          1
3       0         0      1            0          0
4       0         0      1            1          0
5       0         1      0            0          1
6       0         1      0            1          0
7       0         1      1            0          1
8       0         1      1            1          0
9       1         0      0            0          0
10      1         0      0            1          0
11      1         0      1            0          0
12      1         0      1            1          0
13      1         1      0            0          1
14      1         1      0            1          1
15      1         1      1            0          1
16      1         1      1            1          1


<a id="8"></a> <br>
## Predicting New Observations
Now that the tree has been built, there must be a way to classify new observations. The function below will be used to predict the output for the created testing data.

In [13]:
predict_tree = function(testing_data){
  
  # Empty matrix for storing predictions
  predictions = matrix(0, nrow = dim(testing_data)[1])
  
  # Number of inputs
  p = dim(testing_data)[2]
  
  # Make predictions for each row of testing data
  for (i in c(1:(dim(testing_data)[1]))){
    
    # Check each row in the tree
    for (j in c(1:dim(tree_output)[1])){
  
      # Exit loop if the inputs in the tree line-up with the testing data
      if (sum(tree_output[j,][,1:p] == testing_data[i,]) == p){
        break
      }
      
    }
    
    # Store prediction in the kth element of predictions matrix
    predictions[i] = as.numeric(tree_output[j, p+1])
    
  }
  
  # Return the predictions
  return(predictions)
}

<a id="9"></a> <br>
### Simulate Testing Data
Testing data will be created below to predict new observations.

In [14]:
# Number of observations/rows in data
n = 10

# Inputs
exercises = sample(c(0,1), replace = TRUE, size = n)
drinks = sample(c(0,1), replace = TRUE, size = n)
smokes = sample(c(0,1), replace = TRUE, size = n)
relationship = sample(c(0,1), replace = TRUE, size = n)

# Create data frame
testing_data = as.data.frame(cbind(drinks, exercises, smokes, relationship))

# Test predictions
predictions = predict_tree(testing_data)

# Print the predictions
print(predictions)

      [,1]
 [1,]    1
 [2,]    1
 [3,]    1
 [4,]    0
 [5,]    1
 [6,]    0
 [7,]    1
 [8,]    1
 [9,]    0
[10,]    0


<a id="10"></a> <br>
## References
1. https://blog.quantinsti.com/gini-index/